In [1]:
import torch
import numpy as np

import _game
import _bot_b2

import importlib
importlib.reload(_game)
importlib.reload(_bot_b2)

from _game import Game, Game_Hokom, Game_Sun, Bot_Random


In [2]:
models_b2 = [_bot_b2.Model(i) for i in range(4)]
cp_b2 = torch.load("model_b2/cp111000.pt")
[models_b2[i].load_state_dict(cp_b2["models_state_dict"][i]) for i in range(4)]
bots_b2 = [_bot_b2.Bot(models_b2) for i in range(4)]

FileNotFoundError: [Errno 2] No such file or directory: 'model_b2/cp111000.pt'

In [ ]:
game = Game_Hokom(verbose=1)
game.register_bots(bots=bots_b2)
game.whole_game()

hokum: ♠
seat 1 played ♦J (♥8 ♥10 ♥J ♣10 ♣Q ♦10 ♦K)
seat 2 played ♦A (♠10 ♥7 ♥K ♥A ♣7 ♣K ♣A)
seat 3 played ♦8 (♠7 ♠8 ♠J ♠K ♣8 ♣9 ♦7)
seat 0 played ♦9 (♠9 ♠Q ♠A ♥9 ♥Q ♣J ♦Q)
* [[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
Round:1 score:13 Scores:13
seat 2 played ♣K (♠10 ♥7 ♥K ♥A ♣7 ♣A)
seat 3 played ♣9 (♠7 ♠8 ♠J ♠K ♣8 ♦7)
seat 0 played ♣J (♠9 ♠Q ♠A ♥9 ♥Q ♦Q)
seat 1 played ♣10 (♥8 ♥10 ♥J ♣Q ♦10 ♦K)
* [[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
Round:2 score:-16 Scores:-3
seat 1 played ♥10 (♥8 ♥J ♣Q ♦10 ♦K)
seat 2 played ♥A (♠10 ♥7 ♥K ♣7 ♣A)
seat 3 played ♠7 (♠8 ♠J ♠K ♣8 ♦7)
seat 0 played ♥Q (♠9 ♠Q ♠A ♥9 ♦Q)
* [[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 0 0]]
Round:3 score:-24 Scores:-27
seat 3 played ♦7 (♠8 ♠J ♠K ♣8)
seat 0 played ♦Q (♠9 ♠Q ♠A ♥9)
seat 1 played ♦10 (♥8 ♥J ♣Q ♦K)
seat 2 played ♠10 (♥7 ♥K ♣7 ♣A)
* [[0 0 0 0]
 [0 0 0 0]
 [0 0 0 1]
 [0 1 0 0]]
Round:4 score:23 Scores:-4
seat 2 played ♣A (♥7 ♥K ♣7)
seat 3 played ♣8 (♠8 ♠J ♠K)
seat 0 played ♠Q (♠9 ♠A ♥9)
seat 1 played ♣Q (♥8 ♥J ♦K)

In [ ]:
game = Game_Hokom(verbose=1)
game.register_bots(bots=[_bot_b2.Bot(models_b2), Bot_Random(), _bot_b2.Bot(models_b2), Bot_Random()])
game.whole_game()

hokum: ♠
Bot 3 played ♠K (♠7 ♠J ♥J ♥K ♣7 ♣K ♣A)
Bot 0 played ♠8 (♠A ♥A ♣J ♣Q ♦7 ♦8 ♦J)
Bot 1 played ♠9 (♠10 ♥7 ♥10 ♣8 ♣9 ♦9 ♦Q)
Bot 2 played ♠Q (♥8 ♥9 ♥Q ♣10 ♦10 ♦K ♦A)
Round:1 score:-21 Scores:-21
Bot 1 played ♣9 (♠10 ♥7 ♥10 ♣8 ♦9 ♦Q)
Bot 2 played ♣10 (♥8 ♥9 ♥Q ♦10 ♦K ♦A)
Bot 3 played ♣A (♠7 ♠J ♥J ♥K ♣7 ♣K)
Bot 0 played ♣Q (♠A ♥A ♣J ♦7 ♦8 ♦J)
Round:2 score:-24 Scores:-45
Bot 3 played ♣7 (♠7 ♠J ♥J ♥K ♣K)
Bot 0 played ♣J (♠A ♥A ♦7 ♦8 ♦J)
Bot 1 played ♣8 (♠10 ♥7 ♥10 ♦9 ♦Q)
Bot 2 played ♥Q (♥8 ♥9 ♦10 ♦K ♦A)
Round:3 score:5 Scores:-40
Bot 0 played ♦J (♠A ♥A ♦7 ♦8)
Bot 1 played ♦Q (♠10 ♥7 ♥10 ♦9)
Bot 2 played ♦A (♥8 ♥9 ♦10 ♦K)
Bot 3 played ♠7 (♠J ♥J ♥K ♣K)
Round:4 score:-16 Scores:-56
Bot 3 played ♣K (♠J ♥J ♥K)
Bot 0 played ♠A (♥A ♦7 ♦8)
Bot 1 played ♠10 (♥7 ♥10 ♦9)
Bot 2 played ♥9 (♥8 ♦10 ♦K)
Round:5 score:25 Scores:-31
Bot 0 played ♦8 (♥A ♦7)
Bot 1 played ♦9 (♥7 ♥10)
Bot 2 played ♦10 (♥8 ♦K)
Bot 3 played ♠J (♥J ♥K)
Round:6 score:-30 Scores:-61
Bot 3 played ♥K (♥J)
Bot 0 played ♥A (♦7)
Bot